In [1]:
# ✅ 环境准备
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models
import matplotlib.pyplot as plt
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# ✅ 数据预处理
# 📚需额外学习：transforms 中的数据增强技巧，x如 RandomResizedCrop、ColorJitter
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

train_path = "../../Data/food-11/training/labeled"
val_path = "../../Data/food-11/validation"

train_dataset = datasets.ImageFolder(train_path, transform=transform)
val_dataset = datasets.ImageFolder(val_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [3]:
# ✅ 构建 CNN 模型
# 📚需额外学习：nn.Sequential 与手动搭建模型结构的差异
# 📚需额外学习：Dropout 与 BatchNorm 的作用
class CNN(nn.Module):
    def __init__(self, num_classes=11):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*32*32, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = CNN().to(device)

In [4]:
# ✅ 设置损失函数与优化器
# 📚需额外学习：FocalLoss 用于处理类别不均衡（高级内容）
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [5]:
# 📚需额外学习：学习率调度器（CosineAnnealingLR、StepLR等）
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [6]:
# ✅ 训练与验证循环（核心逻辑）
def train(model, loader):
    model.train()
    total_loss, correct = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = criterion(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        correct += (pred.argmax(1) == y).sum().item()
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

In [7]:
def validate(model, loader):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            total_loss += loss.item() * x.size(0)
            correct += (pred.argmax(1) == y).sum().item()
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

In [8]:
for epoch in range(50):
    train_loss, train_acc = train(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)
    scheduler.step()
    print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

Epoch 1: Train Acc=0.0938, Val Acc=0.0909
Epoch 2: Train Acc=0.0854, Val Acc=0.1121
Epoch 3: Train Acc=0.0903, Val Acc=0.1030
Epoch 4: Train Acc=0.0844, Val Acc=0.0909
Epoch 5: Train Acc=0.0877, Val Acc=0.0909
Epoch 6: Train Acc=0.0909, Val Acc=0.0909
Epoch 7: Train Acc=0.0906, Val Acc=0.0909
Epoch 8: Train Acc=0.0909, Val Acc=0.0909
Epoch 9: Train Acc=0.0864, Val Acc=0.0909
Epoch 10: Train Acc=0.0838, Val Acc=0.0909
Epoch 11: Train Acc=0.0912, Val Acc=0.0909
Epoch 12: Train Acc=0.0847, Val Acc=0.0909
Epoch 13: Train Acc=0.0909, Val Acc=0.0909
Epoch 14: Train Acc=0.0909, Val Acc=0.0909
Epoch 15: Train Acc=0.0909, Val Acc=0.0909
Epoch 16: Train Acc=0.0909, Val Acc=0.0909
Epoch 17: Train Acc=0.0909, Val Acc=0.0909
Epoch 18: Train Acc=0.0909, Val Acc=0.0909
Epoch 19: Train Acc=0.0912, Val Acc=0.0909
Epoch 20: Train Acc=0.0909, Val Acc=0.0909
Epoch 21: Train Acc=0.0909, Val Acc=0.0909
Epoch 22: Train Acc=0.0909, Val Acc=0.0909
Epoch 23: Train Acc=0.0912, Val Acc=0.0909
Epoch 24: Train Acc=